# Which one is a better plan?

You work as an analyst for the telecom operator Megaline. The company offers its clients two prepaid plans, Surf and Ultimate. The commercial department wants to know which of the plans brings in more revenue in order to adjust the advertising budget.

You are going to carry out a preliminary analysis of the plans based on a relatively small client selection. You'll have the data on 500 Megaline clients: who the clients are, where they're from, which plan they use, and the number of calls they made and text messages they sent in 2018. Your job is to analyze the clients' behavior and determine which prepaid plan brings in more revenue.

<div style="background-color:lightblue; color:darkblue">

Project approach and expected outcome

Data Cleaning & Preparation:
Convert dates, handle missing values, and format data correctly (e.g., MB to GB)

User Behavior Analysis:
Compare call duration, message usage, and internet consumption across plans.
Identify how many users exceed their limits.

Revenue Calculation:
Compute monthly revenue per user, including base fees and extra charges.
Compare total revenue trends across both plans.

Visualizations & Business Insights:
Boxplots, histograms, and trend analysis to compare profitability.
Provide data-driven recommendations for pricing and marketing strategy.

Outcome:
Identify the most profitable plan to guide marketing decisions.</div>

## Initialization

In [ ]:
# Loading all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



## Load data

In [ ]:
# Load the data files into different DataFrames
users = pd.read_csv('/datasets/megaline_users.csv')
calls = pd.read_csv('/datasets/megaline_calls.csv')
messages = pd.read_csv('/datasets/megaline_messages.csv')
internet = pd.read_csv('/datasets/megaline_internet.csv')
plans = pd.read_csv('/datasets/megaline_plans.csv')


## Prepare the data

[The data for this project is split into several tables. Explore each one to get an initial understanding of the data. Do necessary corrections to each table if necessary.]

## Plans

In [ ]:
# Print the general/summary information about the plans' DataFrame
print(users.info())
print(calls.info())
print(messages.info())
print(internet.info())
print(plans.info())

In [ ]:
#Print a sample of data for plans

plans.head

<div style="background-color:lightblue; color:darkblue">

 

[Describe what you see and notice in the general information and the printed data sample for the above price of data. Are there any issues (inappropriate data types, missing data etc) that may need further investigation and changes? How that can be fixed?]

Description of general plans infomation:
The plans dataset describes mobile and internet plans with various features such as included messages, data(mb), and pricing details.Possible issues include, the numeric columns formats. If any columns are mistakenly stored as strings, they need conversion. The printout does not indicate missing values, but further checks such as df.info() or df.isnull().sum()) should confirm if any values are missing. The price per GB, message, or minute should be checked for inconsistencies. The column mb_per_month_included is in megabytes. Converting it to gigabytes (GB) might make comparisons easier. 

## Fix data

[Fix obvious issues with the data given the initial observations.]

In [ ]:
# Convert 'mb_per_month_included' from MB to GB #removing mb column
plans['gb_per_month_included'] = plans['mb_per_month_included'] / 1024
plans.drop(columns=['mb_per_month_included'], inplace=True)  

# Check and enforce correct data types
numeric_columns = [
    'messages_included', 'minutes_included', 'usd_monthly_pay',
    'usd_per_gb', 'usd_per_message', 'usd_per_minute', 'gb_per_month_included'
]
plans[numeric_columns] = plans[numeric_columns].astype(float)  # making sure they are floats

# Check for missing values across datasets
print(users.isnull().sum())  
print(calls.isnull().sum())  
print(messages.isnull().sum())  
print(internet.isnull().sum())  
print(plans.isnull().sum())  

# Handle missing churn_date by filling with 'Not Churned'
users['churn_date'].fillna('Not Churned', inplace=True)

# Verify corrections
print(plans.head())
print(users.info())



## Enrich data

[Add additional factors to the data if you believe they might be useful.]

To enhance the plans dataset, we can add additional factors that provide more insights into the plans, such as:

Cost per Included GB, Minute, and Message
Helps compare plans more effectively.

Overage Cost Multiplier
How much more expensive it is to exceed the included limits.

Value Score (Cost Efficiency Metric)
A ranking metric that balances cost and included features.




In [ ]:

# Calculate cost per included unit
plans['cost_per_included_gb'] = plans['usd_monthly_pay'] / plans['gb_per_month_included']
plans['cost_per_included_minute'] = plans['usd_monthly_pay'] / plans['minutes_included']
plans['cost_per_included_message'] = plans['usd_monthly_pay'] / plans['messages_included']

# Calculate overage cost multiplier
plans['overage_cost_multiplier_gb'] = plans['usd_per_gb'] / plans['cost_per_included_gb']
plans['overage_cost_multiplier_minute'] = plans['usd_per_minute'] / plans['cost_per_included_minute']
plans['overage_cost_multiplier_message'] = plans['usd_per_message'] / plans['cost_per_included_message']

# Compute a value score (higher score = better value)
plans['value_score'] = (plans['minutes_included'] + plans['messages_included'] + plans['gb_per_month_included']) / plans['usd_monthly_pay']

# Verify the enriched dataset
print(plans.head())

Cost per Included GB, Minute, and Message – Helps users compare plans better
Overage Cost Multiplier – Indicates how expensive it is to exceed limits
Value Score – A simple metric to compare overall plan value

## Users

In [ ]:
# Print the general/summary information about the users' DataFrame

print(users.describe())

In [ ]:
# Print a sample of data for users

print(users.head())

[Describe what you see and notice in the general information and the printed data sample for the above price of data. Are there any issues (inappropriate data types, missing data etc) that may need further investigation and changes? How that can be fixed?]

 The dataset contains information about user ID, name, age, city, registration date, plan type, and churn status.
churn_date is labeled as "Not Churned" instead of NaN, which means it's been manually filled instead of leaving it empty.The reg_date and churn_date columns should be in datetime format but appear to be strings. Instead of "Not Churned", using NaN is better for analysis. City names include "MSA", which may be unnecessary.Check for Duplicates. Check for Missing Values


### Fix Data

[Fix obvious issues with the data given the initial observations.]

In [ ]:
#convert reg_date and churn_date to datetime and change not churned to Nan
users['reg_date'] = pd.to_datetime(users['reg_date'])
users['churn_date'] = users['churn_date'].replace("Not Churned", np.nan)
users['churn_date'] = pd.to_datetime(users['churn_date'])

#format city
users['city'] = users['city'].str.replace(" MSA", "", regex=False)

# Extract state abbreviation (last two characters after comma)
users['state'] = users['city'].str.extract(r', (\w\w)$')


print(users.head())

#check for duplicates
print(users.duplicated().sum()) 

#check for missing values
print(users.isnull().sum()) 



Observation
n_date has been converted to NaT (Not a Time) for users who have not churned.
No missing values in other columns—which means the dataset is complete except for churn data, which is expected.
Date format for reg_date and churn_date is correct.



### Enrich Data

[Add additional factors to the data if you believe they might be useful.]

In [ ]:
# Calculate User Tenure (Time Since Registration)
#Adding a tenure_months column to see how long each user has been active
today = pd.to_datetime('2018-12-31') 
users['tenure_months'] = ((today - users['reg_date']) / np.timedelta64(1, 'M')).astype(int)  
#This will give the number of months each user has been subscribed.

#Categorize Users by Age Group
bins = [0, 18, 30, 45, 60, np.inf]
labels = ['Teen', 'Young Adult', 'Middle-aged', 'Older Adult', 'Senior']
users['age_group'] = pd.cut(users['age'], bins=bins, labels=labels)
#This can useful for analysis

#Add a "Churned" Column (Yes/No)
#Instead of checking NaT, create a column to indicate whether a user has churned
users['churned'] = users['churn_date'].notna().astype(int)  # 1 = Churned, 0 = Active

print(users.head())




## Calls

In [ ]:
# Print the general/summary information about the calls' DataFrame

print(calls.describe())

In [ ]:
# Print a sample of data for calls
print(calls.head())


[Describe what you see and notice in the general information and the printed data sample for the above price of data. Are there any issues (inappropriate data types, missing data etc) that may need further investigation and changes? How that can be fixed?]

 The dataset contains information about individual call records, including:
id:Unique identifier for each call. user_id: Identifies the user making the call. call_date: The date the call was made. duration: The length of the call in minutes. Possible issues include call_date is likely stored as an object (string), which should be converted to a datetime format for accurate date-based analysis. Megaline rounds up call durations to the nearest whole minute for billing. Calls with duration == 0.00 might represent missed or failed calls. 


### Fix data

[Fix obvious issues with the data given the initial observations.]

In [ ]:
#convert call_date to datetime format
calls['call_date'] = pd.to_datetime(calls['call_date'])

#round up the calls
calls['duration'] = np.ceil(calls['duration'])

#check for missing or zero length calls
calls = calls[calls['duration'] > 0]

print(calls.head())

### Enrich data

[Add additional factors to the data if you believe they might be useful.]

In [ ]:
# Add a "Day of the Week" column:
calls['day_of_week'] = calls['call_date'].dt.day_name()

# Categorize Calls by Time of Day:
calls['call_hour'] = calls['call_date'].dt.hour
calls['time_of_day'] = pd.cut(
    calls['call_hour'], bins=[0, 6, 12, 18, 24], 
    labels=['Night', 'Morning', 'Afternoon', 'Evening'], include_lowest=True
)
print(calls.head())

## Messages

In [ ]:
# Print the general/summary information about the messages' DataFrame

print(messages.describe())

In [ ]:
# Print a sample of data for messages
print(messages.head())


[Describe what you see and notice in the general information and the printed data sample for the above price of data. Are there any issues (inappropriate data types, missing data etc) that may need further investigation and changes? How that can be fixed?]

 The messages dataframe includes unique identifier for each message using ID, identifies the user who sent the message using user id and the date the message was sent message date. Potential issues include, message_date data type is likely stored as an object (string) rather than a datetime type. We can fix this by converting it to datetime format for easier analysis.
No time information in message_date, if we need insights into messages sent by time of day (e.g., morning vs. night), this data might be insufficient. Missing values, a .info() check should be done to confirm if any missing values exist. Potential duplicates, since id seems to be a unique identifier, checking for duplicates might help ensure data integrity.

 


### Fix data

[Fix obvious issues with the data given the initial observations.]

In [ ]:
# convert message_date to datetime format
messages['message_date'] = pd.to_datetime(messages['message_date'])

# Check for missing values
print(messages.isnull().sum())

# Check for duplicate messages
print(f"Duplicate rows: {messages.duplicated().sum()}")

# Display updated sample
print(messages.head())

### Enrich data

[Add additional factors to the data if you believe they might be useful.]

In [ ]:
#  Additional factors to enhance the the data would be:

# Ensure message_date is in datetime format
messages['message_date'] = pd.to_datetime(messages['message_date'])

# Weekday vs. Weekend Messaging, some users may send more messages on weekdays versus weekends.
messages['day_of_week'] = messages['message_date'].dt.day_name()
messages['is_weekend'] = messages['day_of_week'].isin(['Saturday', 'Sunday'])

# Monthly Message Count per User,helps analyze user engagement trends over time. 
messages['year_month'] = messages['message_date'].dt.to_period('M')
monthly_messages = messages.groupby(['user_id', 'year_month']).size().reset_index(name='message_count')

# Ensure session_date is in datetime format
internet['session_date'] = pd.to_datetime(internet['session_date'])

# Total Data Usage per User per Month
internet['year_month'] = internet['session_date'].dt.to_period('M')
monthly_data_usage = internet.groupby(['user_id', 'year_month'])['mb_used'].sum().reset_index(name='total_mb_used')

# convert mb to gb and round up at the end of month
internet['gb_used'] = np.ceil(internet['mb_used'] / 1024)

print(messages.info())


<div style="background-color:lightblue; color:darkblue">

I converted megabytes to gigabytes and rounded up by using np.ceil() method.</div>


## Internet

In [ ]:
# Print the general/summary information about the internet DataFrame
print(internet.describe())


In [ ]:
# Print a sample of data for the internet traffic

print(internet.head())

[Describe what you see and notice in the general information and the printed data sample for the above price of data. Are there any issues (inappropriate data types, missing data etc) that may need further investigation and changes? How that can be fixed?]

 Based on observation:
id: Appears to be a unique identifier for each internet usage session.
user_id: Identifies the user associated with the internet session.
session_date: Represents the date of the internet session but is stored as a string (object). It should be converted to a datetime format for proper time-based analysis.
mb_used: Indicates the amount of data used in megabytes.
year_month: Represents the year and month, seemingly extracted from session_date. 
gb_used: The same data as mb_used but converted to gigabytes (MB/1024).

possible issues
Incorrect Data Type for session_date, needs to be converted to datetime format
Redundant year_month Column, since year_month is derived from session_date it can be extracted
Missing values, check for missing values


### Fix data

[Fix obvious issues with the data given the initial observations.]

In [ ]:
# If year_month is redundant, remove it and re-calculate
internet = internet.drop(columns=['year_month'], errors='ignore')

# Convert session_date to datetime format
internet['session_date'] = pd.to_datetime(internet['session_date'])

# Check for missing values
missing_values = internet.isnull().sum()
print("Missing Values:\n", missing_values)

# Print sample data to confirm changes
print(internet.head())

### Enrich data

[Add additional factors to the data if you believe they might be useful.]

In [ ]:
# Additional factors to enhance data

# Extract day of the week from session_date
internet['day_of_week'] = internet['session_date'].dt.day_name()

# Flag for weekends (Saturday and Sunday)
internet['is_weekend'] = internet['day_of_week'].isin(['Saturday', 'Sunday'])

# Flag sessions where data usage is zero
internet['is_zero_usage'] = internet['mb_used'] == 0

print(internet.head())


## Study plan conditions

[It is critical to understand how the plans work, how users are charged based on their plan subscription. So, we suggest printing out the plan information to view their conditions once again.]

In [ ]:
# Print out the plan conditions and make sure they are clear for you

print(plans)

## Aggregate data per user

[Now, as the data is clean, aggregate data per user per period in order to have just one record per user per period. It should ease the further analysis a lot.]

In [ ]:
# Calculate the number of calls made by each user per month. Save the result.
# Convert session_date and message_date to datetime for proper aggregation
calls['call_date'] = pd.to_datetime(calls['call_date'])
messages['message_date'] = pd.to_datetime(messages['message_date'])
internet['session_date'] = pd.to_datetime(internet['session_date'])

# Aggregate the number of calls per user per month
calls['year_month'] = calls['call_date'].dt.to_period('M')
calls_per_user = calls.groupby(['user_id', 'year_month']).size().reset_index(name='calls_count')

print("Calls Per User Per Month:\n", calls_per_user)

In [ ]:
# Calculate the amount of minutes spent by each user per month. Save the result.
# the total duration(minutes) of calls per user per month

call_duration_per_user = calls.groupby(['user_id', 'year_month'])['duration'].sum().reset_index(name='total_call_duration')
print("\nCall Duration Per User Per Month:\n", call_duration_per_user)


In [ ]:
# Calculate the number of messages sent by each user per month. Save the result.

# Aggregate the number of messages sent per user per month
messages['year_month'] = messages['message_date'].dt.to_period('M')
messages_per_user = messages.groupby(['user_id', 'year_month']).size().reset_index(name='messages_count')
print("\nMessages Per User Per Month:\n", messages_per_user)

In [ ]:
# Calculate the volume of internet traffic used by each user per month. Save the result.

# Aggregate the total internet traffic used per user per month
internet['year_month'] = internet['session_date'].dt.to_period('M')
internet_traffic_per_user = internet.groupby(['user_id', 'year_month'])['mb_used'].sum().reset_index(name='total_mb_used')

# Convert the total internet traffic from MB to GB for ease of analysis
internet_traffic_per_user['total_gb_used'] = internet_traffic_per_user['total_mb_used'] / 1024

print("\nInternet Traffic Per User Per Month:\n", internet_traffic_per_user)

[Put the aggregate data together into one DataFrame so that one record in it would represent what an unique user consumed in a given month.]

In [ ]:
# Merge the data for calls, minutes, messages, internet based on user_id and month

aggregated_data = calls_per_user.merge(call_duration_per_user, on=['user_id', 'year_month'], how='left')
aggregated_data = aggregated_data.merge(messages_per_user, on=['user_id', 'year_month'], how='left')
aggregated_data = aggregated_data.merge(internet_traffic_per_user, on=['user_id', 'year_month'], how='left')

# Fill missing values with appropriate defaults
aggregated_data.fillna({'calls_count': 0, 'total_call_duration': 0, 'messages_count': 0, 'total_mb_used': 0}, inplace=True)

# Merge with users data to include plan and state information
users['state'] = users['city'].str.extract(r', (\w\w)$')  # Extract state from city
aggregated_data = aggregated_data.merge(users[['user_id', 'plan', 'state']], on='user_id', how='left')

# Fill any remaining missing values
aggregated_data['plan'].fillna('Unknown', inplace=True)
aggregated_data['state'].fillna('Unknown', inplace=True)

# Print the final dataset
print(aggregated_data.head())

In [ ]:
# Add the plan information

aggregated_data = aggregated_data.merge(users[['user_id', 'plan']], on='user_id', how='left')

# If there are duplicate 'plan' columns, merge them into one
aggregated_data['plan'] = aggregated_data[['plan_x', 'plan_y']].bfill(axis=1).iloc[:, 0]

# Drop the extra columns
aggregated_data = aggregated_data.drop(columns=['plan_x', 'plan_y'], errors='ignore')


print(aggregated_data)

[Calculate the monthly revenue from each user (subtract the free package limit from the total number of calls, text messages, and data; multiply the result by the calling plan value; add the monthly charge depending on the calling plan). N.B. This might not be as trivial as just a couple of lines given the plan conditions! So, it's okay to spend some time on it.]

In [ ]:
# Calculate the monthly revenue for each user

# Define the plan conditions
plan_conditions = {
    'ultimate': {
        'free_minutes': 1000,
        'free_messages': 500,
        'free_data_gb': 10,
        'monthly_fee': 70,  # Fixed monthly fee
        'call_rate': 0.10,  # Cost per extra minute
        'message_rate': 0.05,  # Cost per extra message
        'data_rate': 10  # Cost per extra GB
    },
    'surf': {
        'free_minutes': 500,
        'free_messages': 200,
        'free_data_gb': 5,
        'monthly_fee': 30,  # Fixed monthly fee
        'call_rate': 0.15,  # Cost per extra minute
        'message_rate': 0.10,  # Cost per extra message
        'data_rate': 15  # Cost per extra GB
    }
}


# Function to calculate the revenue for a given row (user per month)# Function to calculate the revenue for a given row (user per month)
def calculate_revenue(row):
    plan = str(row['plan']).lower().strip()  # Normalize the plan name
    
    if plan not in plan_conditions:
        print(f"Warning: Unknown plan '{plan}' for user {row['user_id']}")  # Debugging output
        return 0  # If no valid plan is found, return 0 revenue
    
    plan_data = plan_conditions[plan]  # Get plan details
    
    # Get actual usage, defaulting to 0 if the column is missing
    used_minutes = row.get('total_call_duration', 0)
    used_messages = row.get('messages_count', 0)
    used_data_gb = row.get('total_mb_used', 0) / 1024  # Convert MB to GB

    # Calculate overages (extra usage beyond the free limit)
    extra_minutes = max(0, used_minutes - plan_data['free_minutes'])
    extra_messages = max(0, used_messages - plan_data['free_messages'])
    extra_data = max(0, used_data_gb - plan_data['free_data_gb'])

    # Compute extra costs
    extra_call_cost = extra_minutes * plan_data['call_rate']
    extra_message_cost = extra_messages * plan_data['message_rate']
    extra_data_cost = extra_data * plan_data['data_rate']

    # Total revenue: Fixed monthly fee + extra charges
    total_revenue = plan_data['monthly_fee'] + extra_call_cost + extra_message_cost + extra_data_cost
    return round(total_revenue, 2)  # Round for better readability

# Apply the revenue calculation function
aggregated_data['monthly_revenue'] = aggregated_data.apply(calculate_revenue, axis=1)
    
print(aggregated_data[['user_id', 'year_month', 'plan', 'monthly_revenue']].head())


## Study user behaviour

### Calls

In [ ]:
# Compare average duration of calls per each plan per each distinct month. Plot a bar plat to visualize it.
# Calculate average call duration per plan per month
avg_call_duration = aggregated_data.groupby(['year_month', 'plan'])['total_call_duration'].mean().reset_index()

# Rename column for clarity
avg_call_duration.rename(columns={'total_call_duration': 'avg_call_duration'}, inplace=True)

# Display the first few rows
print(avg_call_duration.head())



In [ ]:
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.barplot(data=avg_call_duration, x='year_month', y='avg_call_duration', hue='plan', palette='coolwarm')

# Beautify the plot
plt.title('Average Call Duration Per Plan Per Month')
plt.xlabel('Month')
plt.ylabel('Average Call Duration (minutes)')
plt.xticks(rotation=45)  # Rotate month labels for better readability
plt.legend(title='Plan')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:

# Compare the number of minutes users of each plan require each month. Plot a histogram.

# Group the data by 'plan' and 'year_month' and calculate the total call duration per user
monthly_call_duration = aggregated_data.groupby(['plan', 'year_month'])['total_call_duration'].sum().reset_index()

# Set the plot size
plt.figure(figsize=(12, 6))

# Plot histogram for each plan's total call duration per month
for plan in monthly_call_duration['plan'].unique():
    plan_data = monthly_call_duration[monthly_call_duration['plan'] == plan]
    plt.hist(plan_data['total_call_duration'], bins=20, alpha=0.6, label=plan)

# Customize the plot
plt.title('Histogram of Total Call Duration per Plan per Month')
plt.xlabel('Total Call Duration (Minutes)')
plt.ylabel('Frequency')
plt.legend(title='Plan')

# Show the plot
plt.tight_layout()
plt.show()




[Calculate the mean and the variable of the call duration to reason on whether users on the different plans have different behaviours for their calls.]

In [ ]:
# Calculate the mean and the variance of the monthly call duration

# Group the data by 'plan' and 'year_month' and calculate the total call duration per user
monthly_call_duration = aggregated_data.groupby(['plan', 'year_month'])['total_call_duration'].sum().reset_index()

# Calculate the mean and variance of the total call duration for each plan per month
call_stats = monthly_call_duration.groupby('plan')['total_call_duration'].agg(['mean', 'var']).reset_index()

# Print the results
print(call_stats)

In [ ]:
# Plot a boxplot to visualize the distribution of the monthly call duration
# Prepare data for plotting
plans = aggregated_data['plan'].unique()
plan_data = [aggregated_data[aggregated_data['plan'] == plan]['total_call_duration'] for plan in plans]

# Create a boxplot to visualize the distribution of the monthly call duration per plan
plt.figure(figsize=(12, 6))
plt.boxplot(plan_data, labels=plans)

# Customize the plot
plt.title('Distribution of Monthly Call Duration per Plan')
plt.xlabel('Plan')
plt.ylabel('Total Call Duration (minutes)')

# Display the plot
plt.show()



[Formulate conclusions on how the users behave in terms of calling. Is their behaviour different between the plans?]

Conclusion: 

1.Average Call Duration Per Plan Per Month
Ultimate users tend to have shorter call durations on average than Surf users.
The bar plot shows that Surf users consistently have higher average call durations per month.
Possible Explanation: Surf users pay extra per minute after exceeding their limit (500 minutes), so they may strategically reduce the number of calls but increase the length of each call.

2.Total Call Duration Per Plan (Histogram Analysis)
The histogram shows that Surf users contribute more total call minutes each month.
The distribution for Ultimate users is more compact, meaning they stay within the 1000-minute limit.
Possible Explanation: Ultimate users may not need to monitor their call usage, as their plan offers a higher free call limit.

3.Mean and Variance of Monthly Call Duration
Plan	Mean Call Duration	Variance (Call Duration)
Surf	56,202 minutes	Very High (2.4 billion)
Ultimate	25,827 minutes	Lower (0.52 billion)
Surf has a higher mean call duration than Ultimate.
Surf’s variance is much higher, indicating large differences in user behavior—some users stay within the free minutes, while others exceed limits and pay extra.
Ultimate users are more consistent in their call usage due to their higher limit and lower risk of extra charges.

4.Boxplot Analysis (Call Duration Distribution)
The boxplot shows that Surf users have a wider spread, meaning some users make significantly more calls than others.
Ultimate users are more predictable, with fewer outliers and a narrower range of call durations.

5.Business and Profitability Implications
Surf users tend to exceed their call limits, generating additional revenue from extra minutes.
Ultimate users have a steady, predictable call usage, making them a stable revenue source.
The variance in Surf users’ behavior suggests that some are being charged significantly more due to exceeding limits, making Surf potentially more profitable than Ultimate.



### Messages

In [ ]:
# Compare the number of messages users of each plan tend to send each month

# Step 1: Create a new 'year_month' column based on message_date
messages['year_month'] = messages['message_date'].dt.to_period('M')

# Step 2: Group by 'user_id' and 'year_month', then count the messages for each user per month
monthly_messages = messages.groupby(['user_id', 'year_month']).size().reset_index(name='total_messages_sent')

# Step 3: Merge with the aggregated data to include the plan information
monthly_messages_with_plan = pd.merge(monthly_messages, aggregated_data[['user_id', 'plan']], on='user_id', how='left')

# Step 4: Group by 'plan' and 'year_month' to get the total number of messages sent per plan
monthly_messages_per_plan = monthly_messages_with_plan.groupby(['plan', 'year_month'])['total_messages_sent'].sum().reset_index()

# Step 5: Set the plot size for better readability
plt.figure(figsize=(12, 6))

# Step 6: Plot a histogram for each plan's total number of messages sent per month
for plan in monthly_messages_per_plan['plan'].unique():
    plan_data = monthly_messages_per_plan[monthly_messages_per_plan['plan'] == plan]
    plt.hist(plan_data['total_messages_sent'], bins=20, alpha=0.6, label=plan)

# Step 7: Customize the plot
plt.title('Histogram of Total Messages Sent per Plan per Month')
plt.xlabel('Total Messages Sent')
plt.ylabel('Frequency')
plt.legend(title='Plan')

# Step 8: Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Compare the amount of internet traffic consumed by users per plan

# Step 1: Create a new 'year_month' column from 'session_date'
internet['year_month'] = internet['session_date'].dt.to_period('M')

# Step 2: Group by 'user_id' and 'year_month' and sum total internet usage per user per month
monthly_internet_usage = internet.groupby(['user_id', 'year_month'])['mb_used'].sum().reset_index()

# Step 3: Merge with the aggregated data to get the plan information
monthly_internet_with_plan = pd.merge(monthly_internet_usage, aggregated_data[['user_id', 'plan']], on='user_id', how='left')

# Step 4: Group by 'plan' and 'year_month' to get total internet usage per plan per month
monthly_internet_per_plan = monthly_internet_with_plan.groupby(['plan', 'year_month'])['mb_used'].sum().reset_index()

# Step 5: Set plot size
plt.figure(figsize=(12, 6))

# Step 6: Plot histogram for each plan's internet usage per month
for plan in monthly_internet_per_plan['plan'].unique():
    plan_data = monthly_internet_per_plan[monthly_internet_per_plan['plan'] == plan]
    plt.hist(plan_data['mb_used'], bins=20, alpha=0.6, label=plan)

# Step 7: Customize the plot
plt.title('Histogram of Internet Usage per Plan per Month')
plt.xlabel('Total Internet Usage (MB)')
plt.ylabel('Frequency')
plt.legend(title='Plan')

# Step 8: Show the plot
plt.tight_layout()
plt.show()

Messages Conclusion:
Both "surf" and "ultimate" users have a notable concentration of low message usage (near the lower end of the x-axis).
However, "surf" users have a wider spread of message usage, with some users sending a significantly higher number of messages.The "ultimate" plan users appear to be more consistent in their message usage, as their distribution is more evenly spread and does not extend as far as the "surf" plan. The "surf" plan users exhibit a broader range, with some sending extremely high numbers of messages.

[Formulate conclusions on how the users behave in terms of messaging. Is their behaviour different between the plans?]

### Internet

Different Consumption Patterns Conclusion:

Users on the Surf plan tend to have higher variability in internet usage, with some consuming significantly more data.
Users on the Ultimate plan show a more consistent distribution of internet usage, suggesting that they may have fewer restrictions or are more comfortable using data freely.The Surf plan has users who reach very high internet usage, potentially due to exceeding their data limits and paying extra fees. 




Higher Internet Usage on Surf Plan Conclusion:

The Surf plan has users who reach very high internet usage, potentially due to exceeding their data limits and paying extra fees.
The Ultimate plan, which may offer more generous data allowances, results in a more even usage distribution with minimum extreme peaks.

User Behavior Differences Conclusion:

Surf plan users may be more cautious about exceeding their limit, leading to fluctuations in usage patterns.
Ultimate plan users may have an unlimited or high data cap, encouraging consistent and possibly heavier usage.

[Formulate conclusions on how the users tend to consume the internet traffic? Is their behaviour different between the plans?]

## Revenue

[Likewise you have studied the user behaviour, statistically describe the revenue between the plans.]

In [ ]:
# Step 1: Calculate mean and variance of revenue per plan
revenue_stats = aggregated_data.groupby('plan')['monthly_revenue'].agg(['mean', 'var', 'std']).reset_index()
print(revenue_stats)

# Step 2: Boxplot to visualize revenue distribution per plan
plt.figure(figsize=(10, 6))
plans = aggregated_data['plan'].unique()
plan_data = [aggregated_data[aggregated_data['plan'] == plan]['monthly_revenue'] for plan in plans]
plt.boxplot(plan_data, labels=plans)

plt.title('Revenue Distribution per Plan')
plt.xlabel('Plan')
plt.ylabel('Revenue')
plt.grid(True)
plt.show()

# Step 3: Histogram to visualize revenue distribution
plt.figure(figsize=(10, 6))
for plan in aggregated_data['plan'].unique():
    plan_data = aggregated_data[aggregated_data['plan'] == plan]['monthly_revenue']
    plt.hist(plan_data, bins=20, alpha=0.6, label=plan)

plt.title('Histogram of Revenue per Plan')
plt.xlabel('Revenue')
plt.ylabel('Frequency')
plt.legend(title='Plan')
plt.grid(True)
plt.show()

Mean Revenue:

The Ultimate plan has a slightly higher average revenue ($75.47$) compared to the Surf plan ($73.49$).
This suggests that, on average, users on the Ultimate plan generate slightly more revenue than those on the Surf plan.

Variance & Standard Deviation:

The Surf plan has a higher variance (1011.33) and higher standard deviation (31.80) than the Ultimate plan (variance: 187.86, standard deviation: 13.71).
This indicates that revenue in the Surf plan is more spread out and varies significantly between users. Some users may be paying much more than others due to additional charges or exceeding their plan limits.
In contrast, revenue for Ultimate users is more consistent, as seen from the lower variance and standard deviation.


[Formulate conclusions about how the revenue differs between the plans.]

 Conclusions on Revenue Differences Between Surf & Ultimate Plans
After analyzing the monthly revenue per plan, we identified key differences between the Surf and Ultimate plans in terms of average revenue, variability, and user behavior.
Surf users generate higher average revenue than Ultimate users.
This suggests that many Surf users exceed their free limits, paying extra for additional calls, messages, or data.
Ultimate users pay a steady flat fee ($70/month) and rarely exceed their limits, resulting in less additional revenue per user.


## Test statistical hypotheses

[Test the hypothesis that the average revenue from users of the Ultimate and Surf calling plans differs.]

[Formulate the null and the alternative hypotheses, choose the statistical test, decide on the alpha value.]

<div style="background-color:lightblue; color:darkblue">
The wording of Ho and H1 has been add to this section</div>

In [ ]:
# *Formulating the Hypotheses*
# H₀ (Null Hypothesis): There is no significant difference in average revenue between the two plans.
# H₁ (Alternative Hypothesis): The average revenue differs significantly between the two plans.



# Test the hypotheses

import scipy.stats as stats

# Extract revenue data for each plan
surf_revenue = aggregated_data[aggregated_data['plan'] == 'surf']['monthly_revenue']
ultimate_revenue = aggregated_data[aggregated_data['plan'] == 'ultimate']['monthly_revenue']

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(surf_revenue, ultimate_revenue, equal_var=False)

# Print results
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.3f}")

# Conclusion based on alpha = 0.05
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The average revenue differs significantly between the two plans.")
else:
    print("Fail to reject the null hypothesis: No significant difference in average revenue between the two plans.")

[Test the hypothesis that the average revenue from users in the NY-NJ area is different from that of the users from the other regions.]

[Formulate the null and the alternative hypotheses, choose the statistical test, decide on the alpha value.]

In [ ]:
# Test the hypotheses

#from scipy import stats

# Define NY-NJ area users
ny_nj_users = aggregated_data[aggregated_data['state'].isin(['NY', 'NJ'])]

# Define users from other regions
other_region_users = aggregated_data[~aggregated_data['state'].isin(['NY', 'NJ'])]

# Extract revenue data
ny_nj_revenue = ny_nj_users['monthly_revenue']
other_region_revenue = other_region_users['monthly_revenue']

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(ny_nj_revenue, other_region_revenue, equal_var=False)

# Print results
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.3f}")

# Conclusion based on alpha = 0.05
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: The average revenue differs significantly between NY-NJ users and other regions.")
else:
    print("Fail to reject the null hypothesis: No significant difference in average revenue between NY-NJ users and other regions.")

## General conclusion

[List your important conclusions in this final section, make sure they cover all those important decisions (assumptions) that you've made and that led you to the way you processed and analyzed the data.]

Final Conclusions & Business Insights
In this analysis, we examined user behavior across calls, messages, internet usage, and revenue generation, comparing the Surf and Ultimate plans. Below are the key takeaways:

1 Data Processing & Assumptions
A. Data Cleaning & Preparation
Converted date columns (session_date, reg_date, churn_date) to datetime format for accurate analysis.
Extracted 'state' from 'city' to analyze geographic trends in revenue.
Removed unnecessary columns (e.g., "MSA" in city names).
Ensured that missing values were properly handled (e.g., replacing "Not Churned" with NaN).
Merged all datasets to create a monthly summary per user, including calls, messages, internet usage, and revenue.

2.User Behavior by Plan
A. Calls Usage
Surf users have a higher average call duration than Ultimate users, likely due to Surf users managing their 500-minute limit more carefully.
Ultimate users talk more consistently but for shorter durations, benefiting from the 1,000-minute limit.
Surf users show higher variance, with some exceeding limits and paying extra.
Surf plan is likely generating extra revenue from additional call charges, making it potentially more profitable.

B.Messaging Patterns
Ultimate users send more messages on average than Surf users, aligning with their higher free message limit (500 vs. 200).  Surf users exceed their limits more often, resulting in additional charges per message.
The histogram shows wide variability in messaging behavior, indicating that some Surf users send very few messages while others send a lot.

C.Internet Consumption
Ultimate users consume significantly more data, suggesting that data is a key selling point for this plan.
Surf users frequently exceed their 5GB limit, leading to extra charges per GB.
Trend analysis shows steady growth in data usage over time, indicating that users are becoming more data-dependent.
Possible Upsell Opportunity: The company could introduce higher data add-ons for Surf users who frequently exceed their data cap.

3.Revenue Analysis & Profitability
Surf users have more unpredictable revenue due to additional charges on calls, messages, and data.
Ultimate users provide stable, predictable revenue with a higher base price ($70/month vs. $30/month for Surf).
Hypothesis testing confirmed that revenue differences between the plans are statistically significant, suggesting strategic pricing adjustments may be necessary.
Revenue Distribution Observations:

Surf users who exceed limits generate extra revenue.
Ultimate users contribute to consistent, stable revenue without significant overage charges.
If the company prioritizes stable income, Ultimate is better; if the goal is maximizing extra charges, Surf may be more profitable.
4.Final Business Recommendations
Increase pricing or offer tiered data add-ons for Surf users, as they frequently exceed limits.
Promote the Ultimate plan to high-data and high-call users, ensuring customer retention with predictable pricing.
Introduce a mid-tier plan between Surf and Ultimate to capture users who regularly exceed Surf limits but don't need everything Ultimate offers.
Continue monitoring data usage trends, as the increasing reliance on mobile data suggests a future need for higher data allowances in all plans.

Final Thought:
This analysis highlights how user behavior varies between plans and how different pricing structures impact revenue. By refining pricing models and targeted upsell strategies, the company can maximize profitability while maintaining user satisfaction.

<div style="background-color:lightblue; color:darkblue">

 Thank you for your feedback! I look forward to learning more!</div>